In [24]:
import xarray as xr

In [25]:
ds = xr.open_dataset("/Users/stefanlippl/dev/wef_final/spatial-risk-mapping/noaa/s3_downloads/20210322_gfs.t12z.pgrb2.0p25.f024", engine="pynio")

In [27]:
print(len(ds))
for v in ds:
    print("{},\t {},\t {}".format(v, ds[v].attrs["long_name"], ds[v].attrs["units"]))

178
TMP_P0_L1_GLL0,	 Temperature,	 K
TMP_P0_L6_GLL0,	 Temperature,	 K
TMP_P0_L7_GLL0,	 Temperature,	 K
TMP_P0_L100_GLL0,	 Temperature,	 K
TMP_P0_L102_GLL0,	 Temperature,	 K
TMP_P0_L103_GLL0,	 Temperature,	 K
TMP_P0_L104_GLL0,	 Temperature,	 K
TMP_P0_2L108_GLL0,	 Temperature,	 K
TMP_P0_L109_GLL0,	 Temperature,	 K
POT_P0_L104_GLL0,	 Potential temperature,	 K
DPT_P0_L103_GLL0,	 Dew point temperature,	 K
APTMP_P0_L103_GLL0,	 Apparent temperature,	 K
SPFH_P0_L100_GLL0,	 Specific humidity,	 kg kg-1
SPFH_P0_L103_GLL0,	 Specific humidity,	 kg kg-1
SPFH_P0_2L108_GLL0,	 Specific humidity,	 kg kg-1
RH_P0_L4_GLL0,	 Relative humidity,	 %
RH_P0_L100_GLL0,	 Relative humidity,	 %
RH_P0_L103_GLL0,	 Relative humidity,	 %
RH_P0_2L104_GLL0,	 Relative humidity,	 %
RH_P0_L104_GLL0,	 Relative humidity,	 %
RH_P0_2L108_GLL0,	 Relative humidity,	 %
RH_P0_L200_GLL0,	 Relative humidity,	 %
RH_P0_L204_GLL0,	 Relative humidity,	 %
PWAT_P0_L200_GLL0,	 Precipitable water,	 kg m-2
PRATE_P0_L1_GLL0,	 Precipitation rate

In [28]:
print(ds["VGRD_P0_L103_GLL0"])

<xarray.DataArray 'VGRD_P0_L103_GLL0' (lv_HTGL7: 7, lat_0: 721, lon_0: 1440)>
[7267680 values with dtype=float32]
Coordinates:
  * lv_HTGL7  (lv_HTGL7) float32 10.0 20.0 30.0 40.0 50.0 80.0 100.0
  * lat_0     (lat_0) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon_0     (lon_0) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
Attributes:
    center:                                         US National Weather Servi...
    production_status:                              Operational products
    long_name:                                      V-component of wind
    units:                                          m s-1
    grid_type:                                      Latitude/longitude
    parameter_discipline_and_category:              Meteorological products, ...
    parameter_template_discipline_category_number:  [0 0 2 3]
    level_type:                                     Specified height level ab...
    forecast_time:                                  [24]
 

In [29]:
ds["VGRD_P0_L103_GLL0"].to_dataframe()

VGRD_P0_L103_GLL0
lv_HTGL7 lat_0 lon_0                    
10.0      90.0 0.00            -3.777748
               0.25            -3.777748
               0.50            -3.787748
               0.75            -3.787748
               1.00            -3.787748
...                                  ...
100.0    -90.0 358.75          -9.362493
               359.00          -9.372493
               359.25          -9.382492
               359.50          -9.402493
               359.75          -9.412493

[7267680 rows x 1 columns]

In [30]:
ds = ds.get("VGRD_P0_L103_GLL0")

In [31]:
df = ds.to_dataframe()
df

VGRD_P0_L103_GLL0
lv_HTGL7 lat_0 lon_0                    
10.0      90.0 0.00            -3.777748
               0.25            -3.777748
               0.50            -3.787748
               0.75            -3.787748
               1.00            -3.787748
...                                  ...
100.0    -90.0 358.75          -9.362493
               359.00          -9.372493
               359.25          -9.382492
               359.50          -9.402493
               359.75          -9.412493

[7267680 rows x 1 columns]

In [32]:
latitudes = df.index.get_level_values("lat_0")
longitudes = df.index.get_level_values("lon_0")

In [33]:
map_function = lambda lon: (lon - 360) if (lon > 180) else lon
remapped_longitudes = longitudes.map(map_function)

In [34]:
df["longitude"] = remapped_longitudes
df["latitude"] = latitudes

In [35]:
min_lon = 15.0
max_lon = 33.0
min_lat = -35.0
max_lat = -22.0

In [36]:
lat_filter = (df["latitude"] >= min_lat) & (df["latitude"] <= max_lat)
lon_filter = (df["longitude"] >= min_lon) & (df["longitude"] <= max_lon)

In [37]:
df = df.loc[lat_filter & lon_filter]

In [38]:
df

VGRD_P0_L103_GLL0  longitude  latitude
lv_HTGL7 lat_0 lon_0                                        
10.0     -22.0 15.00          -0.997748      15.00     -22.0
               15.25          -2.117748      15.25     -22.0
               15.50           0.782252      15.50     -22.0
               15.75          -0.537748      15.75     -22.0
               16.00          -2.617748      16.00     -22.0
...                                 ...        ...       ...
100.0    -35.0 32.00           4.537507      32.00     -35.0
               32.25           4.517507      32.25     -35.0
               32.50           4.697507      32.50     -35.0
               32.75           4.867507      32.75     -35.0
               33.00           5.097507      33.00     -35.0

[27083 rows x 3 columns]

In [39]:
df.reset_index(inplace=True)

In [40]:
df

,lv_HTGL7,lat_0,lon_0,VGRD_P0_L103_GLL0,longitude,latitude
0,10.0,-22.0,15.00,-0.997748,15.00,-22.0
1,10.0,-22.0,15.25,-2.117748,15.25,-22.0
2,10.0,-22.0,15.50,0.782252,15.50,-22.0
3,10.0,-22.0,15.75,-0.537748,15.75,-22.0
4,10.0,-22.0,16.00,-2.617748,16.00,-22.0
...,...,...,...,...,...,...
27078,100.0,-35.0,32.00,4.537507,32.00,-35.0
27079,100.0,-35.0,32.25,4.517507,32.25,-35.0
27080,100.0,-35.0,32.50,4.697507,32.50,-35.0
27081,100.0,-35.0,32.75,4.867507,32.75,-35.0


In [41]:
df = df.query('lv_HTGL7 == 10.0')

In [42]:
df.lv_HTGL2.unique()

AttributeError: 'DataFrame' object has no attribute 'lv_HTGL2'

In [44]:
df = df[['longitude', 'latitude', 'VGRD_P0_L103_GLL0']]

In [45]:
df

,longitude,latitude,VGRD_P0_L103_GLL0
0,15.00,-22.0,-0.997748
1,15.25,-22.0,-2.117748
2,15.50,-22.0,0.782252
3,15.75,-22.0,-0.537748
4,16.00,-22.0,-2.617748
...,...,...,...
3864,32.00,-35.0,4.392252
3865,32.25,-35.0,4.372252
3866,32.50,-35.0,4.542252
3867,32.75,-35.0,4.702252


In [22]:
import plotly.express as px

fig = px.scatter(df, x="longitude", y="latitude", color="tmp_celsius")
fig.show()

ModuleNotFoundError: No module named 'plotly'